# Tagging and Extraction Using OpenAI functions

In [1]:
import os

import openai
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

# Code of your application, which uses environment variables (e.g. from `os.environ` or
# `os.getenv`) as if they came from the actual environment.
# openai.api_key = os.environ['OPENAI_API_KEY']

True

In [2]:
# from typing import List
# from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from langchain_core.utils.function_calling import convert_to_openai_function
from pydantic import BaseModel, Field

In [3]:
class Tagging(BaseModel):
    """Tag the piece of text with particular info."""

    sentiment: str = Field(
        description="sentiment of text, should be `pos`, `neg`, or `neutral`"
    )
    language: str = Field(description="language of text (should be ISO 639-1 code)")

In [4]:
convert_to_openai_function(Tagging)

{'name': 'Tagging',
 'description': 'Tag the piece of text with particular info.',
 'parameters': {'type': 'object',
  'properties': {'sentiment': {'description': 'sentiment of text, should be `pos`, `neg`, or `neutral`',
    'type': 'string'},
   'language': {'description': 'language of text (should be ISO 639-1 code)',
    'type': 'string'}},
  'required': ['sentiment', 'language']}}

In [5]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [6]:
model = ChatOpenAI(temperature=0)

In [7]:
tagging_functions = [convert_to_openai_function(Tagging)]

In [8]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Think carefully, and then tag the text as instructed"),
        ("user", "{input}"),
    ]
)

In [9]:
model_with_functions = model.bind(
    functions=tagging_functions, function_call={"name": "Tagging"}
)

In [10]:
tagging_chain = prompt | model_with_functions

In [11]:
tagging_chain.invoke({"input": "I love langchain"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"sentiment":"pos","language":"en"}', 'name': 'Tagging'}}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 108, 'total_tokens': 118}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6d7243c4-3d01-479d-9cc4-2181e07839dc-0', usage_metadata={'input_tokens': 108, 'output_tokens': 10, 'total_tokens': 118})

In [12]:
tagging_chain.invoke({"input": "non mi piace questo cibo"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"sentiment":"neg","language":"it"}', 'name': 'Tagging'}}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 111, 'total_tokens': 121}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6442d0a3-0e43-4171-80e9-f8c1a8fc355b-0', usage_metadata={'input_tokens': 111, 'output_tokens': 10, 'total_tokens': 121})

In [13]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

In [14]:
tagging_chain = prompt | model_with_functions | JsonOutputFunctionsParser()

In [15]:
tagging_chain.invoke({"input": "non mi piace questo cibo"})

{'sentiment': 'neg', 'language': 'it'}

In [16]:
tagging_chain.invoke(
    {"input": "I have to go back to work tomorrow. School holidays are over."}
)

{'sentiment': 'neg', 'language': 'en'}

In [17]:
tagging_chain.invoke(
    {"input": "I have to go back to work tomorrow. School holidays are too short."}
)

{'sentiment': 'neg', 'language': 'en'}

## Extraction

Extraction is similar to tagging, but used for extracting multiple pieces of information.

In [18]:
from typing import Optional


class Person(BaseModel):
    """Information about a person."""

    name: str = Field(description="person's name")
    age: Optional[int] = Field(description="person's age")

class Information(BaseModel):
    """Information to extract."""

    people: list[Person] = Field(description="List of info about people")

In [27]:
pip install pydantic==1.10.2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 225.1 kB/s eta 0:00:0000:0100:01
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.7.4
    Uninstalling pydantic-2.7.4:
      Successfully uninstalled pydantic-2.7.4
Note: you may need to restart the kernel to use updated packages.


In [19]:
convert_to_openai_function(Information)

{'name': 'Information',
 'description': 'Information to extract.',
 'parameters': {'type': 'object',
  'properties': {'people': {'description': 'List of info about people',
    'type': 'array',
    'items': {'description': 'Information about a person.',
     'type': 'object',
     'properties': {'name': {'description': "person's name", 'type': 'string'},
      'age': {'description': "person's age", 'type': 'integer'}},
     'required': ['name']}}},
  'required': ['people']}}

In [20]:
extraction_functions = [convert_to_openai_function(Information)]
extraction_model = model.bind(
    functions=extraction_functions, function_call={"name": "Information"}
)

In [21]:
extraction_model.invoke("Joe is 30, his mom is Martha")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"people":[{"name":"Joe","age":30},{"name":"Martha"}]}', 'name': 'Information'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 93, 'total_tokens': 110}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f622a4eb-9282-4412-82a4-8a88ead8da72-0', usage_metadata={'input_tokens': 93, 'output_tokens': 17, 'total_tokens': 110})

In [22]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Extract the relevant information, if not explicitly provided do not guess. Extract partial info",
        ),
        ("human", "{input}"),
    ]
)

In [23]:
extraction_chain = prompt | extraction_model

In [24]:
extraction_chain.invoke({"input": "Joe is 30, his mom is Martha"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"people":[{"name":"Joe","age":30},{"name":"Martha"}]}', 'name': 'Information'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 110, 'total_tokens': 127}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b70cbf9c-0a5c-48c9-b6f5-2650805b8705-0', usage_metadata={'input_tokens': 110, 'output_tokens': 17, 'total_tokens': 127})

In [25]:
extraction_chain = prompt | extraction_model | JsonOutputFunctionsParser()

In [26]:
extraction_chain.invoke({"input": "Joe is 30, his mom is Martha"})

{'people': [{'name': 'Joe', 'age': 30}, {'name': 'Martha'}]}

In [27]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

In [28]:
extraction_chain = (
    prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="people")
)

In [29]:
extraction_chain.invoke({"input": "Joe is 30, his mom is Martha"})

[{'name': 'Joe', 'age': 30}, {'name': 'Martha'}]

## Doing it for real

We can apply tagging to a larger body of text.

For example, let's load this blog post and extract tag information from a sub-set of the text.

In [30]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
documents = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [31]:
len(documents)

1

In [32]:
doc = documents[0]

In [33]:
page_content = doc.page_content[:10000]

In [34]:
print(page_content[:1000])







LLM Powered Autonomous Agents | Lil'Log







































Lil'Log






















Posts




Archive




Search




Tags




FAQ




emojisearch.app









      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


 


Table of Contents



Agent System Overview

Component One: Planning

Task Decomposition

Self-Reflection


Component Two: Memory

Types of Memory

Maximum Inner Product Search (MIPS)


Component Three: Tool Use

Case Studies

Scientific Discovery Agent

Generative Agents Simulation

Proof-of-Concept Examples


Challenges

Citation

References





Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general

In [35]:
class Overview(BaseModel):
    """Overview of a section of text."""

    summary: str = Field(description="Provide a concise summary of the content.")
    language: str = Field(
        description="Provide the language that the content is written in."
    )
    keywords: str = Field(description="Provide keywords related to the content.")

In [36]:
overview_tagging_function = [convert_to_openai_function(Overview)]
tagging_model = model.bind(
    functions=overview_tagging_function, function_call={"name": "Overview"}
)
tagging_chain = prompt | tagging_model | JsonOutputFunctionsParser()

In [37]:
tagging_chain.invoke({"input": page_content})

{'summary': 'The article discusses building autonomous agents powered by LLM (large language model) as the core controller. It covers components like planning, memory, and tool use, along with challenges and references.',
 'language': 'English',
 'keywords': 'LLM, autonomous agents, planning, memory, tool use, challenges, references'}

In [38]:
from pydantic import BaseModel, Field


class Paper(BaseModel):
    """Information about papers mentioned."""

    title: str
    author: Optional[str]


class Info(BaseModel):
    """Information to extract"""

    papers: list[Paper]

In [39]:
convert_to_openai_function(Info)

{'name': 'Info',
 'description': 'Information to extract',
 'parameters': {'type': 'object',
  'properties': {'papers': {'type': 'array',
    'items': {'description': 'Information about papers mentioned.',
     'type': 'object',
     'properties': {'title': {'type': 'string'}, 'author': {'type': 'string'}},
     'required': ['title']}}},
  'required': ['papers']}}

### Manual fixed the wrong openai function

In [40]:
openai_function_manual = {
    "name": "Info",
    "description": "Information to extract",
    "parameters": {
        "type": "object",
        "properties": {
            "papers": {
                "type": "array",
                "items": {
                    "description": "Information about papers mentioned.",
                    "type": "object",
                    "properties": {
                        "title": {"type": "string"},
                        "author": {"type": "string"},
                    },
                    "required": ["title", "author"],
                },
            }
        },
        "required": ["papers"],
    },
}

In [41]:
# paper_extraction_function = [convert_to_openai_function(Info)]
paper_extraction_function = [openai_function_manual]
extraction_model = model.bind(
    functions=paper_extraction_function, function_call={"name": "Info"}
)
extraction_chain = (
    prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="papers")
)

In [42]:
extraction_chain.invoke({"input": page_content})

[{'title': 'LLM Powered Autonomous Agents', 'author': 'Lilian Weng'}]

In [43]:
template = """A article will be passed to you. Extract from it all papers that are mentioned by this article. 

Do not extract the name of the article itself. If no papers are mentioned that's fine - you don't need to extract any! Just return an empty list.

Do not make up or guess ANY extra information. Only extract what exactly is in the text.

Make sure the title of paper is included."""

prompt = ChatPromptTemplate.from_messages([("system", template), ("human", "{input}")])

In [44]:
extraction_chain = (
    prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="papers")
)

In [45]:
extraction_chain.invoke({"input": page_content})

[{'title': 'Chain of thought (CoT; Wei et al. 2022)'},
 {'title': 'Tree of Thoughts (Yao et al. 2023)'},
 {'title': 'LLM+P (Liu et al. 2023)'},
 {'title': 'ReAct (Yao et al. 2023)'},
 {'title': 'Reflexion (Shinn & Labash 2023)'},
 {'title': 'Chain of Hindsight (CoH; Liu et al. 2023)'},
 {'title': 'Algorithm Distillation (AD; Laskin et al. 2023)'}]

In [46]:
extraction_chain.invoke({"input": "hi"})

[]

In [47]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_overlap=0)

In [48]:
splits = text_splitter.split_text(doc.page_content)

In [49]:
len(splits)

15

In [50]:
def flatten(matrix):
    flat_list = []
    for row in matrix:
        flat_list += row
    return flat_list

In [51]:
flatten([[1, 2], [3, 4]])

[1, 2, 3, 4]

In [52]:
print(splits[0])

LLM Powered Autonomous Agents | Lil'Log







































Lil'Log






















Posts




Archive




Search




Tags




FAQ




emojisearch.app









      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


 


Table of Contents



Agent System Overview

Component One: Planning

Task Decomposition

Self-Reflection


Component Two: Memory

Types of Memory

Maximum Inner Product Search (MIPS)


Component Three: Tool Use

Case Studies

Scientific Discovery Agent

Generative Agents Simulation

Proof-of-Concept Examples


Challenges

Citation

References





Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general probl

In [53]:
from langchain.schema.runnable import RunnableLambda

In [54]:
prep = RunnableLambda(lambda x: [{"input": doc} for doc in text_splitter.split_text(x)])

In [55]:
prep.invoke("hi")

[{'input': 'hi'}]

In [56]:
chain = prep | extraction_chain.map() | flatten

In [57]:
chain.invoke(doc.page_content)

[{'title': 'Maximum Inner Product Search (MIPS)',
  'author': 'Y. Shen and W. Wang'},
 {'title': 'Scientific Discovery Agent', 'author': 'J. Smith and A. Johnson'},
 {'title': 'Generative Agents Simulation', 'author': 'K. Lee and M. Chen'},
 {'title': 'Chain of thought', 'author': 'Wei et al. 2022'},
 {'title': 'Tree of Thoughts', 'author': 'Yao et al. 2023'},
 {'title': 'LLM+P', 'author': 'Liu et al. 2023'},
 {'title': 'ReAct', 'author': 'Yao et al. 2023'},
 {'title': 'Reflexion', 'author': 'Shinn & Labash 2023'},
 {'title': 'Chain of Hindsight', 'author': 'Liu et al. 2023'},
 {'title': 'Algorithm Distillation', 'author': 'Laskin et al. 2023'},
 {'title': 'Laskin et al. 2023'},
 {'title': 'Miller 1956'},
 {'title': 'Duan et al. 2017'},
 {'title': 'LSH: Locality-Sensitive Hashing',
  'author': 'Andoni, Alexandr and Indyk, Piotr'},
 {'title': 'ANNOY: Approximate Nearest Neighbors Oh Yeah',
  'author': 'Bernhardsson, Erik'},
 {'title': 'HNSW: Hierarchical Navigable Small World',
  'autho